In [6]:
import pandas as pd
import syft as sy
import os
import dotenv
import time
import sqlite3
import hashlib

Skip to the WikiReddit Query section if you do not have access to / want to skip the data collecction from Reddit.

# Requerying the Reddit API

In [2]:
# we recommend that you source these values using environment variables
dotenv.load_dotenv()
URL = "https://reddit-for-researchers.snooguts.net"
EMAIL = os.getenv("EMAIL")
PASSWORD = os.getenv("PASSWORD")

# you can provide a "password" keyword argument, but if you don't...
# the browser will prompt you for input
client = sy.login(
    url=URL,
    email=EMAIL,
    password=PASSWORD
)

Logged into <reddit-for-researchers-internal: High side Datasite> as <prgildersleve@gmail.com>


In [4]:
def name_to_id_dict(client):
    # TODO handle case with duplicate names?
    return {request.code.service_func_name: str(request.id) for request in client.requests}


def query_r4r(function_name, name_to_id_dict, client):

    while True:   
        try:
            request = client.api.services.request.get_by_uid(uid=sy.UID(name_to_id_dict[function_name]))
            job = request.code(blocking=False)
            results = job.wait()
            df = results.get()
            return df
        except KeyboardInterrupt:
            raise
        except Exception as ex:
            print(ex)
            time.sleep(10)
            client = sy.login(
                url=URL,
                email=EMAIL,
                password=PASSWORD
            )

In [ ]:
# annnoyingly this takes a while when one has many queries
# Recommend using a cache, should probably implement this in the future
funcdict = name_to_id_dict(client)
funcdict

In [ ]:
# The pg_wiki_202312 query gets all posts from December 2023 that mention wikipedia
# """
# SELECT *
# FROM posts 
# WHERE (body LIKE '%wikipedia%' OR title LIKE '%wikipedia%')
# AND created_at >= '2023-12-01'
# AND created_at < '2024-01-01'
# """

test_data = query_r4r('pg_wiki_202312', funcdict, client)

In [28]:
keepcols = ['id', 'crosspost_parent_id', 'subreddit_id', 'title', 'body',
       'score', 'upvote_ratio', 'language_code', 'gildings',
       'num_comments', 'created_at', 'updated_at']
anon_test_data = test_data[keepcols].copy()

# fill the title col and body col with random text - only to obscure the data in this demo
def anonymise_key(key):
    if key:
        return hashlib.sha256(key.encode()).hexdigest()
    else:
        return key

anon_test_data['title'] = anon_test_data['title'].apply(anonymise_key)
anon_test_data['body'] = anon_test_data['body'].apply(anonymise_key)

# Apply hashes to the ID cols, this allows one to query the WikiReddit database for the data

anon_test_data = anon_test_data.rename({'id': 'post_id'}, axis=1)
anon_test_data['post_id'] = anon_test_data['post_id'].apply(anonymise_key)
anon_test_data['crosspost_parent_id'] = anon_test_data['crosspost_parent_id'].apply(anonymise_key)
anon_test_data['subreddit_id'] = anon_test_data['subreddit_id'].apply(anonymise_key)

# Convert the data types to the correct types
anon_test_data = anon_test_data.astype({'score': int, 'gildings': int, 'num_comments': int})
anon_test_data['created_at'] = pd.to_datetime(anon_test_data['created_at']).astype('datetime64[ns]')
anon_test_data['updated_at'] = pd.to_datetime(anon_test_data['updated_at']).astype('datetime64[ns]')
anon_test_data.to_hdf('requery_demo_data.h5', key='df', mode='w')

# Here is our mock data
anon_test_data

/tmp/ipykernel_741669/3530956112.py:27: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['post_id', 'crosspost_parent_id', 'subreddit_id', 'title', 'body',
       'language_code'],
      dtype='object')]

  anon_test_data.to_hdf('requery_demo_data.h5', key='df', mode='w')


,post_id,crosspost_parent_id,subreddit_id,title,body,score,upvote_ratio,language_code,gildings,num_comments,created_at,updated_at
0,71524bd1624760572af3188cce22b22a636b216af2637d...,None,be8b6202826cfbc5d172fd8d919657884e023e701f7252...,f701ade3eff95c4618d797fcba3f9d562db4059c104606...,562397dc6fb6af4adc0441dbe52dd0071adbb997f9a7c3...,9,1.000000,en,0,6,2023-12-15 18:54:25.676,2023-12-19 17:36:27.785829
1,0b12d8735be95487a7d5e098ebba16a172a6af48938e3b...,None,30bede6b71ce82be9aef9e4ad0238a30b7a74d0ad7d5a5...,cda9c9e4dce2a2db72eb5c0fbca82f1054be92092f48a0...,ad5bb1c449c3a346e409887b1c9dfaf438838b695a4cd1...,1,0.533333,en,0,4,2023-12-15 11:26:09.028,2023-12-16 03:59:39.774596
2,c5c6f74e8f6b172b68055bab975a878d577b9e594f177f...,None,0c0b7d5846a828306e804957bf548ed7d1365cd4cc6117...,1207d4d47f7839212ec0fca64b003ba2741b370c73bf89...,47fae7f19534ace92f6786f91c5cde183d7440c41b7432...,-43,0.258427,en,0,37,2023-12-16 01:53:24.225,2023-12-16 19:50:13.491670
3,36340d0c846d70c4a8015fe05ddb2311f8e6371fff6218...,None,5a89004cb1daf9b3b53c162e5ac26b7644fd336a8d4c96...,1293d5c8c38026884a251f90cc1fb23578054a25760e21...,3d7e06a9a787ea4487d4fb40072c6d7ba60d73fb2b5641...,33,0.883721,en,0,9,2023-12-22 23:41:16.804,2023-12-24 13:29:45.712510
4,59d047b7106494e47522b93ce524e287a52ecacbe5ccd3...,None,0971e2934285338cc753dd10dc664bd7bbca80ac06b414...,5544dfe5a077ccf21ce49e248dfef906f9e21146ba8c7d...,a5e03807f9b2b3e8209b9432e36658b17e61646bc2df51...,-1,0.333333,en,0,17,2023-12-03 02:41:32.790,2023-12-03 08:50:40.322282
...,...,...,...,...,...,...,...,...,...,...,...,...
5744,6b833bdc81c78181d9b9080c9fd93996877827bb134d35...,None,822e057f8f6a2728b7efa9ac2e0d00adea064ed4a3e347...,66e11effedb836a5ff66585cdf511640a41ec320cc8f91...,b19a2f113941f37492a18c9a6c8c3868465cae44b97650...,29,1.000000,en,0,16,2023-12-04 05:04:51.260,2024-04-01 05:40:34.032969
5745,dfa5ce27e4baba6205e1539cd00af0a760303b0013fc9e...,None,2f515a57a4000934d0431495e98c8203d242de56d2a890...,0c9d908dbc35a354c556012261740e6052d84bd230b6a1...,dc23926058c37a00853ab63f5c18012603c142b3d321e6...,164,0.960674,en,0,40,2023-12-19 21:07:38.307,2024-03-23 03:31:02.871910
5746,a4d46f34ab2e076cb073c447bfd5a9c27861739587e416...,None,ac21f49f12aa02f03608d471c28141a9cc549687e99191...,025e380e4aaf37e3ba3da65ad2e7dea208409a8c43832d...,e331e34b82472171ca13e0c58767e1a175560cb336e9f2...,1,1.000000,en,0,0,2023-12-11 00:28:25.328,2023-12-11 00:28:25.359451
5747,703fb56a8b817472fad7f8e8fc4fbea73b8e7b13320eba...,None,29259bcc98d1de3a5ed46be1c36d12d57bb7949da6c387...,8ed165b7763aac60a79f07a10bf5faab667bad8823676b...,9cf7e39618061fb1d9c7d224ea3041176cef2b0d596dd5...,29,1.000000,en,0,21,2023-12-01 09:37:24.041,2023-12-16 22:14:06.539576


## WikiReddit Query

In [31]:
# read the anon test data with fake post title and text

anon_test_data = pd.read_hdf('requery_demo_data.h5', key='df')

In [40]:
# open wikireddit database

conn = sqlite3.connect('wikireddit.db')
postlinks = pd.read_sql_query("SELECT * FROM postlinks", conn)
linkarticles = pd.read_sql_query("SELECT * FROM linkarticles", conn)

In [42]:
# get the post links and Wikipedia articlesfor the test data

anon_test_data_with_wrdata = anon_test_data.merge(postlinks, on='post_id')
anon_test_data_with_wrdata = anon_test_data_with_wrdata.merge(linkarticles, on='final_url')
anon_test_data_with_wrdata[['post_id', 'title', 'body', 'final_url', 'raw_title']]

# One can now continue with analysis of articles, pageviews, revisions, etc,
# together with the freshly extracted post text (anonymized in this demo) from the API

,post_id,title,body,final_url,raw_title
0,71524bd1624760572af3188cce22b22a636b216af2637d...,f701ade3eff95c4618d797fcba3f9d562db4059c104606...,562397dc6fb6af4adc0441dbe52dd0071adbb997f9a7c3...,https://en.m.wikipedia.org/wiki/Wikipedia:Wiki...,Wikipedia:WikiProject Articles for creation/He...
1,c5c6f74e8f6b172b68055bab975a878d577b9e594f177f...,1207d4d47f7839212ec0fca64b003ba2741b370c73bf89...,47fae7f19534ace92f6786f91c5cde183d7440c41b7432...,https://en.wikipedia.org/wiki/Promotion_and_re...,Promotion and relegation
2,36340d0c846d70c4a8015fe05ddb2311f8e6371fff6218...,1293d5c8c38026884a251f90cc1fb23578054a25760e21...,3d7e06a9a787ea4487d4fb40072c6d7ba60d73fb2b5641...,https://en.m.wikipedia.org/wiki/Long-term_nucl...,Long-term nuclear waste warning messages
3,59d047b7106494e47522b93ce524e287a52ecacbe5ccd3...,5544dfe5a077ccf21ce49e248dfef906f9e21146ba8c7d...,a5e03807f9b2b3e8209b9432e36658b17e61646bc2df51...,https://en.m.wikipedia.org/wiki/List_of_Egypti...,List of Egyptian hieroglyphs
4,9269197dbb5cf90e908856509570aa0a1aa0eb154ed9b1...,5b841e39f47eae9a89f316c45866b93381a9b2f2d8ab11...,1499e41809686ef63860782bd9c3c4c6c03d3241850161...,https://en.m.wikipedia.org/wiki/Wholeness_and_...,Wholeness and the Implicate Order
...,...,...,...,...,...
10638,dfa5ce27e4baba6205e1539cd00af0a760303b0013fc9e...,0c9d908dbc35a354c556012261740e6052d84bd230b6a1...,dc23926058c37a00853ab63f5c18012603c142b3d321e6...,https://en.wikipedia.org/wiki/August_Bier,August Bier
10639,a4d46f34ab2e076cb073c447bfd5a9c27861739587e416...,025e380e4aaf37e3ba3da65ad2e7dea208409a8c43832d...,e331e34b82472171ca13e0c58767e1a175560cb336e9f2...,https://en.wikipedia.org/wiki/Demographics_of_...,Demographics of Dubai
10640,703fb56a8b817472fad7f8e8fc4fbea73b8e7b13320eba...,8ed165b7763aac60a79f07a10bf5faab667bad8823676b...,9cf7e39618061fb1d9c7d224ea3041176cef2b0d596dd5...,https://en.wikipedia.org/wiki/Prefectures_of_J...,Prefectures of Japan
10641,6a7274a7cc4778e63078eaf3e7d5a71a1f366cadc130d6...,cf29e94dee37d3e4977c9464f1aa695cf06f09a9772f75...,0f3bb7a123d4b2a5917c07ae0cc87b59e2702a0c82c74a...,https://en.wikipedia.org/wiki/Vahagn,Vahagn
